In [2]:
import madxmodule
import numpy as np
import pandas as pd

import time
s1 = time.time()
# setting bump amplitude
bumpamplist = [i for i in np.arange(-0.0005,0.005,0.0005)]
npart = 1600


basefilelist=[
   [ 'lhcb2-twiss-noerrb-0p5.tfs','lhcb2-twiss-mqxalignfieldb-0p5.tfs'],
    [ 'lhcb2-twiss-noerrb0p0.tfs','lhcb2-twiss-mqxalignfieldb0p0.tfs'],
    [ 'lhcb2-twiss-noerrb0p5.tfs','lhcb2-twiss-mqxalignfieldb0p5.tfs'],
    [ 'lhcb2-twiss-noerrb1p0.tfs','lhcb2-twiss-mqxalignfieldb1p0.tfs'],
    [ 'lhcb2-twiss-noerrb1p5.tfs','lhcb2-twiss-mqxalignfieldb1p5.tfs'],
    [ 'lhcb2-twiss-noerrb2p0.tfs','lhcb2-twiss-mqxalignfieldb2p0.tfs'],
    [ 'lhcb2-twiss-noerrb2p5.tfs','lhcb2-twiss-mqxalignfieldb2p5.tfs'],
    [ 'lhcb2-twiss-noerrb3p0.tfs','lhcb2-twiss-mqxalignfieldb3p0.tfs'],
    [ 'lhcb2-twiss-noerrb3p5.tfs','lhcb2-twiss-mqxalignfieldb3p5.tfs'],
    [ 'lhcb2-twiss-noerrb4p0.tfs','lhcb2-twiss-mqxalignfieldb4p0.tfs'],
    [ 'lhcb2-twiss-noerrb4p5.tfs','lhcb2-twiss-mqxalignfieldb4p5.tfs']
]

errorstringlist=[
        '',
        '''
USE, PERIOD=LHCB2;

EOPTION,ADD=TRUE,SEED=62971100;

SELECT, FLAG=ERROR, PATTERN="MQX*.*L5";
EALIGN, DX:=TGAUSS(1.0)*5.0E-5,DY:=TGAUSS(1.0)*5.0E-5,
        DS:=TGAUSS(1.0)*5.0E-5;
        
ERR = TGAUSS(1.0)*1.0E-4;

SELECT, FLAG=ERROR, PATTERN="MQX*.*L5";
EFCOMP, ORDER:=1, RADIUS=0.010,
    DKNR={0,ERR,ERR*10.,ERR*10.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0},
    DKSR={0,ERR,ERR*10.,ERR*10.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0};
        
        ''']

# correctorlist
corrlist = np.array(['MCBCH.7L5.B2', 'MCBCH.9L5.B2', 'MCBH.13L5.B2'])

# twiss file list
basetwissfilelist = [[madxmodule.Twiss('lhcb2',basefilelist[j][i],targetxc=bumpamplist[j],targetel="MQ.11L5.B2",
                        correctorlist=corrlist,errorseq=errorstringlist[i],
                        twisscols=madxmodule.MADtwissColumns["LHCTwiss"],
                        beam=[madxmodule.MADX_Beam(1,seq='LHCB1',ey=1.5e-6),
                              madxmodule.MADX_Beam(2,seq='LHCB2',ey=1.5e-6)])
                    for i in range(len(basefilelist[j]))] for j in range(len(basefilelist))]
# initial parameters for BFPP
initialdictlist = [[madxmodule.get_initial(fn,madxmodule.dpPb(0,-1)) for fn in basetwissfilelist[i]] 
                   for i in range(len(basetwissfilelist))]  

fileextlist = [['noerrb-0p5','mqxalignfieldb-0p5'],
               ['noerrb0p0','mqxalignfieldb0p0'],
               ['noerrb0p5','mqxalignfieldb0p5'],
               ['noerrb1p0','mqxalignfieldb1p0'],
               ['noerrb1p5','mqxalignfieldb1p5'],
               ['noerrb2p0','mqxalignfieldb2p0'],
               ['noerrb2p5','mqxalignfieldb2p5'],
               ['noerrb3p0','mqxalignfieldb3p0'],
               ['noerrb3p5','mqxalignfieldb3p5'],
               ['noerrb4p0','mqxalignfieldb4p0'],
               ['noerrb4p5','mqxalignfieldb4p5']
        
]

transfermatrixfilelist = [
    [madxmodule.TransferMatrix('LHCB2','IP5','S.DS.L5.B2',initialdictlist[j][i],bumpamplist[j],"MQ.11L5.B2",corrlist,
                              errorseq=errorstringlist[i],fileext=fileextlist[j][i],
                              beam=[madxmodule.MADX_Beam(1,seq='LHCB1',ey=1.5e-6),
                                    madxmodule.MADX_Beam(2,seq='LHCB2',ey=1.5e-6)]
                             ) for i in range(len(basetwissfilelist[j]))] for j in range(len(basefilelist))]

# writing transfermatrixfilelist to file for importing and plotting
dftransfermatrixfilelist       = [pd.DataFrame(pd.Series(data=transfermatrixfilelist[j]),columns=['filename']) 
                                  for j in range(len(transfermatrixfilelist))]
for j in range(len(dftransfermatrixfilelist)):
    dftransfermatrixfilelist[j].to_csv('transfermatrixfilenamelist'+ str(j) +'.csv',index=False)
# calculating impact points and writing them to files + generating file with list of filenames
opt         = madxmodule.io.tfsDict(basetwissfilelist[0][0])
prevelinex  = opt[0]["name"].index("MB.B11L5.B2")-1
s0          = opt[0]['s'][prevelinex]
print s0
# generating the filenames for storing the impactdistributions
impactfilelist    = [['impactIP5left_bp22_' + ext for ext in fileextlist[j]] for j in range(len(fileextlist))]
impactfilelistext = [[fn+'.csv' for fn in impactfilelist[j]] for j in range(len(impactfilelist))]

dfimpactfilelist  = [pd.DataFrame(pd.Series(data=impactfilelistext[j]),columns=['filename']) 
                     for j in range(len(impactfilelist))]
for j in range(len(dfimpactfilelist)):
    dfimpactfilelist[j].to_csv('impactfilenamelist' + str(j) +'.csv',index=False)
# generating the impact distributions
for j in range(len(transfermatrixfilelist)):
    for i in range(len(transfermatrixfilelist[j])):
        madxmodule.impactcoordinates6D(madxmodule.TrackSigmaMatrix(transfermatrixfilelist[j][i],
                                                                   'MB.B11L5.B2',initialdictlist[j][i],npart)
                        ,madxmodule.beampipediam,madxmodule.lhcradius,s0,madxmodule.dipolelength,
                                       madxmodule.dpPb(0,-1),madxmodule.ionmass,
                                       -madxmodule.electronmassgev,
                                       madxmodule.get_p(transfermatrixfilelist[j][i]),
                                       impactfilelist[j][i],beam4=True)
s2 = time.time()
print s2-s1

403.8427102
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
578.421772957
